In [1]:
import time
import config
import numpy as np
from tqdm import tqdm
import os
from datetime import datetime
from database import Database


from model import SentenceEmbedding
from update import update_clusters
from message_kafka import send_message
from cluster_algorithm import DBSCAN_algorithm
import argparse
from dotenv import load_dotenv

load_dotenv()

def countdown_timer(seconds):
    """
    Hàm đếm thời gian từ giá trị giây được cung cấp.

    Args:
        seconds (int): Số giây để đếm ngược.
    """
    while seconds > 0:
        mins, secs = divmod(seconds, 60)
        time_format = '{:02d}:{:02d}'.format(mins, secs)
        print(time_format, end='\r')
        time.sleep(1)
        seconds -= 1
    print("Time's up!")

def ConfigCluster(numPost):
    min_sample = 4 if numPost >= 1550 else 3
    esp = (
        6 if numPost <= 900 else
        5.75 if numPost <= 1200 else
        5.5 if numPost <= 1500 else 5
    )
    return min_sample, esp

def Merge(dict1, dict2):

    return {**dict1, **dict2}

def check_time_used_little(time_now):
    time_now = time_now.strftime('%H:%M')
    time_slots = [
        datetime.strptime("00:00", "%H:%M").strftime('%H:%M'),
        datetime.strptime("07:00", "%H:%M").strftime('%H:%M'),
        datetime.strptime("20:00", "%H:%M").strftime('%H:%M'),
        datetime.strptime("23:59", "%H:%M").strftime('%H:%M')
    ]

    return (time_slots[0] <= time_now < time_slots[1]) or (time_slots[2] <= time_now < time_slots[3])

def update_none_activate(posts, list_index, DB):
    DB.delete_datanotincluster()
    time.sleep(5)
    for idx in tqdm(range(len(posts))):
        if idx not in list_index:
            DB.vn_newflow["datanotinclusters"].insert_one(posts[idx])

f:\TinThoiSu\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load model vnCoreNLP


In [2]:
print("hello")

#load model sentence embedding folder model
model_path = config.MODEL_PATH
model = SentenceEmbedding(model_path)
print("jkkk")
    
    #get time now
timeSystem = datetime.now()
print(timeSystem)

#call Class DB
DB = Database(config, timeSystem)
print("hel")

#get data from database:
dataCluster, dataArticle = DB.get_all_article()
print("num post:", len(dataCluster))


dataClear = [[text[0], text[1], text[2]] for text in dataCluster]
id_articles, embeddings, titlePapers = model.get_embedding(dataClear)

min_sample, esp = ConfigCluster(len(dataCluster))

cluster = DBSCAN_algorithm(esp=esp, min_sample=min_sample, vector_embedding=embeddings)

uniqueCluster, counts = np.unique(cluster, return_counts=True)
uniqueCluster = [idx for idx in uniqueCluster if idx != -1]

clusters_outputs = {}
list_index_not_activate = []
for idx in uniqueCluster:
    clusters_outputs[idx] = []
    for id in range(len(cluster)):
        if cluster[id] == idx:
            clusters_outputs[idx].append(id)
            list_index_not_activate.append(id)



2025-01-02 11:24:12,378 - INFO - Use pytorch device_name: cpu
2025-01-02 11:24:12,379 - INFO - Load pretrained SentenceTransformer: VoVanPhuc/sup-SimCSE-VietNamese-phobert-base


hello


2025-01-02 11:24:12,702 - WARNING - No sentence-transformers model found with name VoVanPhuc/sup-SimCSE-VietNamese-phobert-base. Creating a new one with mean pooling.


jkkk
2025-01-02 11:24:14.185199
hel
Duplicate article found, count:  1
Duplicate article found, count:  2
Duplicate article found, count:  3
Duplicate article found, count:  4
Duplicate article found, count:  5
Duplicate article found, count:  6
Duplicate article found, count:  7
Duplicate article found, count:  8
Duplicate article found, count:  9
Duplicate article found, count:  10
Duplicate article found, count:  11
Duplicate article found, count:  12
Duplicate article found, count:  13
Duplicate article found, count:  14
Duplicate article found, count:  15
Duplicate article found, count:  16
Duplicate article found, count:  17
Duplicate article found, count:  18
Duplicate article found, count:  19
Duplicate article found, count:  20
Duplicate article found, count:  21
Duplicate article found, count:  22
Duplicate article found, count:  23
Duplicate article found, count:  24
Duplicate article found, count:  25
Duplicate article found, count:  26
Duplicate article found, count:  27
D

100%|██████████| 716/716 [01:53<00:00,  6.29it/s]


In [3]:
dataClear = [[text[0], text[1], text[2]] for text in dataCluster]
id_articles, embeddings, titlePapers = model.get_embedding(dataClear)

min_sample, esp = ConfigCluster(len(dataCluster))

cluster = DBSCAN_algorithm(esp=esp, min_sample=min_sample, vector_embedding=embeddings)


100%|██████████| 716/716 [01:52<00:00,  6.34it/s]


In [ ]:
cluster = DBSCAN_algorithm(esp=esp, min_sample=min_sample, vector_embedding=embeddings)

In [4]:
uniqueCluster, counts = np.unique(cluster, return_counts=True)
uniqueCluster = [idx for idx in uniqueCluster if idx != -1]

clusters_outputs = {}
list_index_not_activate = []
for idx in uniqueCluster:
    clusters_outputs[idx] = []
    for id in range(len(cluster)):
        if cluster[id] == idx:
            clusters_outputs[idx].append(id)
            list_index_not_activate.append(id)


In [6]:
print("num cluster:", len(clusters_outputs))
print(clusters_outputs)

num cluster: 42
{np.int64(0): [1, 32, 33, 47, 56, 57, 59, 71, 91, 114, 137, 146, 151, 168, 181, 183, 235, 237, 278, 293, 301, 333, 383, 389, 393, 439, 451, 492, 533, 537, 556, 635, 638, 643, 675, 680, 705], np.int64(1): [4, 35, 96, 196, 233, 256, 340, 390, 448, 453, 469, 475, 570, 613, 615, 692, 710], np.int64(2): [5, 40, 43, 44, 45, 63, 66, 83, 123, 124, 127, 128, 129, 130, 138, 139, 140, 147, 166, 170, 171, 185, 186, 188, 189, 217, 220, 239, 242, 246, 248, 260, 261, 262, 264, 280, 282, 283, 286, 287, 319, 320, 321, 337, 339, 370, 392, 395, 396, 397, 398, 399, 409, 423, 430, 435, 459, 484, 487, 488, 491, 503, 513, 524, 530, 541, 543, 546, 551, 566, 568, 586, 588, 592, 594, 619, 625, 629, 630, 631, 652, 653, 661, 662, 691, 694], np.int64(3): [6, 8, 164, 394, 485], np.int64(4): [7, 11, 516], np.int64(5): [0, 10, 49, 73, 95, 106, 176, 385, 623], np.int64(6): [13, 143, 257, 296, 444, 465, 468, 482, 559], np.int64(7): [14, 29, 84, 86, 89, 153, 154, 174, 211, 212, 215, 216, 222, 275, 380, 4

In [18]:
import time
import config
import numpy as np
from tqdm import tqdm
import os
from datetime import datetime
from database import Database
from model import SentenceEmbedding
from update import update_clusters
from message_kafka import send_message
from cluster_algorithm import DBSCAN_algorithm
import argparse

In [19]:
def update_none_activate(posts, list_index, DB):
    DB.delete_datanotincluster()
    time.sleep(5)
    for idx in tqdm(range(len(posts))):
        try:
            if idx not in list_index:
                DB.vn_newflow["datanotinclusters"].insert_one(posts[idx])
        except Exception as e:
            print(f"Error inserting post at index {idx}:", e)
            print(f"Post data: {posts[idx]}")


In [20]:
update_none_activate(dataArticle, list_index_not_activate, DB)

start delete:


TypeError: unsupported operand type(s) for +: 'int' and 'str'